# Q1

# Q2

A função `train` é responsável pelo treinamento do modelo ao longo de várias épocas. No início de cada época, o modelo é configurado para o modo de treinamento para ativar funcionalidades específicas, como dropout e normalização em lote, que são úteis durante o treinamento. Em seguida, itera-se sobre os lotes de dados de treinamento, calculando a perda entre as previsões do modelo e os rótulos reais. Essa perda é então retropropagada através do modelo para calcular os gradientes e, posteriormente, os parâmetros do modelo são atualizados com base nesses gradientes, utilizando o otimizador especificado. Isso é feito para todos os lotes de treinamento em cada época. Após o término de uma época, a perda média de treinamento é calculada e registrada, e o modelo é avaliado no conjunto de validação usando a função `evaluate`. A perda de validação é calculada e registrada também. Em seguida, o progresso do treinamento é impresso, exibindo a época atual, a perda média de treinamento e a perda de validação correspondente. Esse processo é repetido para o número especificado de épocas.

Já a função `evaluate` é usada para avaliar o modelo no conjunto de validação após cada época de treinamento. O modelo é configurado no modo de avaliação para desativar funcionalidades específicas, como dropout, que são úteis apenas durante o treinamento. Em seguida, itera-se sobre os lotes de dados de validação, passando-os pelo modelo para obter as previsões. A perda entre essas previsões e os rótulos reais é calculada e acumulada. Após percorrer todos os lotes de validação, a média das perdas é calculada e retornada como a perda de validação total.

In [ ]:
import torch

# Função de treinamento
def train(model, criterion, optimizer, train_loader, val_loader, epochs=100):
    # Lista para armazenar as perdas de treinamento e validação ao longo das épocas
    train_losses = []
    val_losses = []

    # Iteração sobre as épocas de treinamento
    for epoch in range(epochs):
        # Coloca o modelo no modo de treinamento
        model.train()
        # Inicializa a perda acumulada durante o treinamento desta época
        running_loss = 0.0

        # Iteração sobre os lotes de treinamento
        for inputs, labels in train_loader:
            # Zera os gradientes acumulados
            optimizer.zero_grad()
            # Passa os dados de entrada pelo modelo para obter as previsões
            outputs = model(inputs)
            # Calcula a perda entre as previsões e os rótulos reais
            loss = criterion(outputs, labels.unsqueeze(1))
            # Retropropagação: calcula os gradientes da perda em relação aos parâmetros do modelo
            loss.backward()
            # Atualiza os parâmetros do modelo com base nos gradientes calculados
            optimizer.step()
            # Acumula a perda atual
            running_loss += loss.item()
        
        # Calcula a perda média de treinamento para esta época
        train_loss = running_loss / len(train_loader)
        # Adiciona a perda média de treinamento à lista de perdas de treinamento
        train_losses.append(train_loss)

        # Avalia o modelo no conjunto de validação
        val_loss = evaluate(model, criterion, val_loader)
        # Adiciona a perda de validação à lista de perdas de validação
        val_losses.append(val_loss)
        
        # Imprime o progresso do treinamento
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

    # Retorna as listas de perdas de treinamento e validação
    return train_losses, val_losses


# Função de avaliação
def evaluate(model, criterion, val_loader):
    # Coloca o modelo no modo de avaliação
    model.eval()
    # Inicializa a perda acumulada durante a avaliação
    running_loss = 0.0

    # Temporariamente desabilita o cálculo do gradiente para economizar memória durante a avaliação
    with torch.no_grad():
        # Iteração sobre os lotes de validação
        for inputs, labels in val_loader:
            # Passa os dados de entrada pelo modelo para obter as previsões
            outputs = model(inputs)
            # Calcula a perda entre as previsões e os rótulos reais
            loss = criterion(outputs, labels.unsqueeze(1))
            # Acumula a perda atual
            running_loss += loss.item()
    
    # Retorna a média das perdas de validação
    return running_loss / len(val_loader)


## a) (os outros são iguais)

Este código faz o seguinte:

- Divide os dados em conjuntos de treino, validação e teste usando `train_test_split`.
- Converte os dados para tensores do PyTorch.
- Define a arquitetura de uma rede neural simples com uma camada oculta usando a classe `PerceptronXOR`.
- Instancia o modelo, a função de perda (no caso, Binary Cross-Entropy Loss) e o otimizador (Adam).
- Define o tamanho do lote (batch) e cria os DataLoader para os conjuntos de treino e validação.
- Treina o modelo usando a função `train` definida em utils, que itera sobre os dados de treino e realiza a retropropagação.
- Avalia o modelo nos dados de teste usando a função `evaluate`.
- Calcula a acurácia do modelo nos dados de teste

In [ ]:
# Importar as bibliotecas necessárias
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from utils import train, evaluate  # Supondo que essas funções estejam definidas em um módulo chamado utils

# Dividir os dados em conjuntos de treino, validação e teste
x_train, x_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.2, random_state=10)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=10)

# Converter os dados para tensores do PyTorch
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_val_tensor = torch.tensor(x_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Definir a arquitetura da rede neural
class PerceptronXOR(nn.Module):
    def __init__(self):
        super(PerceptronXOR, self).__init__()
        # Camada de entrada com 2 neurônios e camada oculta com 4 neurônios
        self.fc1 = nn.Linear(2, 4)
        # Camada de saída com 1 neurônio
        self.fc2 = nn.Linear(4, 1)

    def forward(self, x):
        # Ativação ReLU na camada oculta
        x = torch.relu(self.fc1(x))
        # Ativação sigmoide na camada de saída para obter probabilidades
        x = torch.sigmoid(self.fc2(x))
        return x

# Instanciar o modelo da rede neural, função de perda e otimizador
model = PerceptronXOR()
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Otimizador Adam com taxa de aprendizado de 0.001

# Definir o tamanho do lote (batch) e criar os DataLoader para os conjuntos de treino e validação
batch_size = 32
train_loader = DataLoader(TensorDataset(x_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(x_val_tensor, y_val_tensor), batch_size=batch_size)

# Treinar o modelo utilizando a função definida em utils.train
train_losses, val_losses = train(model, criterion, optimizer, train_loader, val_loader, epochs=100)

# Avaliar o modelo nos dados de teste
test_loader = DataLoader(TensorDataset(x_test_tensor, y_test_tensor), batch_size=batch_size)
test_loss = evaluate(model, criterion, test_loader)
print(f"\nTest Loss: {test_loss:.4f}")

# Calcular a acurácia do modelo
model.eval()  # Mudar para o modo de avaliação
with torch.no_grad():  # Desativar o cálculo de gradientes durante a avaliação
    outputs = model(x_test_tensor)
    predicted = torch.round(outputs)  # Arredondar as saídas para 0 ou 1
    accuracy = (predicted == y_test_tensor.unsqueeze(1)).sum().item() / predicted.size(0)  # Calcular a acurácia
    print(f"Acurácia: {accuracy:.2f}")
